# Canvas Batch Change Dates

This script allows you to adjust the start and end times of multiple courses at once in Canvas. At each stage you will be asked to give certain inputs and asked confirmation steps before moving forward. Please read through all of the steps and confirm that you have all of the information needed.

If any of the steps along the way fail, the script will not move forward. There should be indicators of why the failure occurred and steps to take to fix the errors.

### To Run (once you have read all of the instructions):
**Kernel -> Restart & Run All**

## 1. Token Validation
This is the token validation stage. If you do not enter a valid token, nothing else will happen.

In [1]:
from canvas_batch_change_dates import initialize

initialize()

Enter Canvas API Token: ········

Token Valid

Hello Alison


## 1. b) Create a course file from subaccount

TO BE IMPLEMENTED

## 2. Fill out Input CSV
1. You should have a csv called `start_end_courses-template.csv` in `data/input` that has the columns **course_id**, **start_date**, and **end_date**.
1. Create a copy of this file called `start_end_courses.csv` and add your course information to it. Please leave the template file empty.
1. Please ensure the dates are formatted as **YYYY-MM-DD** or **YYYY-MM-DD HH:MM** in your file. If you have used Excel, the columns should be formatted as a "Short Date" (YYYY-MM-DD) or "Long Date" (YYYY-MM-DD HH:MM). 
    * The script assumes that the dates are for Vancouver
    * The start_date will assume the time of start is 12:00am that day
    * The end_date will assume the time of end is 11:59pm that day


Once you have completed the script, `start_end_courses.csv` will be copied to the `/completed` folder and timestamped in the filename.


## 3. Set Participation Dates

The default setting for restricting participations to course dates is **None**. This will keep whatever setting was initially established. You can change this value below to **True** or **False** 

**None** will keep settings as is. 
**True** will restrict participation to within the given dates.
**False** will allow participation outside of the given dates. 

In [2]:
# restrict_participation_to_dates can be True, False, or None
restrict_participation_to_dates = True

## 4. Apply the Changes
In this step you will be asked to confirm the csv that is being read, and to confirm the start and end date changes that will occur.
You should see indicators for success or failure for each of the courses as the script runs.

In [3]:
from datetime import datetime
from canvas_batch_change_dates import get_courses_df, create_course_update_df

# NOW is the time you made the update, not the time the csv is output
NOW = datetime.now()

courses_df = get_courses_df('data/input/start_end_courses.csv')

details = create_course_update_df(courses_df, restrict_participation_to_dates)

print('\nTotal Courses: {}\n'.format(len(details)))
display(details)

Successfully loaded csv file


,course_id,course_name,note,start_at_og,start_at_new,end_at_og,end_at_new,restrict_enrol_og,restrict_enrol_new
0,10456,Alison Myers Sandbox,OK,2018-09-21T18:49:42Z,2015-09-01 12:00:00 AM PDT-0700,None,2032-01-01 11:59:59 PM PST-0800,False,True


Would you like to try to update the above? (y/n)
y

10456 course updated successfully

Total Courses: 1



,course_id,course_name,change_success,change_message,note,start_at_og,start_at_new,end_at_og,end_at_new,restrict_enrol_og,restrict_enrol_new
0,10456,Alison Myers Sandbox,True,course updated,OK,2018-09-21T18:49:42Z,2015-09-01 12:00:00 AM PDT-0700,None,2032-01-01 11:59:59 PM PST-0800,False,True


## 5. Generate Final Output
The final output will be a csv with the changes that have taken place (if confirmed in the step above).

In [4]:
from canvas_batch_change_dates import output_csv

output_csv(details)

Details:
    CSV NAME: "canvas_course_settings_2021-01-12 14.18.51.csv""
    OUTPUT FOLDER: "data/output"

Do you want to generate the csv with these details? (y/n): y

"canvas_course_settings_2021-01-12 14.18.51.csv" CREATED IN "data/output"

Goodbye!
